In [1]:
import numpy as np

with open('./data/coef_model2.npy', 'rb') as f:
    coef = np.load(f)

coef_NKI = coef[:, 0]
coef_share = coef[:, 1]

In [2]:
import gzip
import pickle

filename = './data/slider_values.pkl.gz'

with gzip.open(filename, 'rb') as f:
    X_df = pickle.load(f)
    
filename = './data/outcomes.pkl.gz'

with gzip.open(filename, 'rb') as f:
    Y_df = pickle.load(f)

In [29]:
import ipywidgets as widgets

knowledge_init = X_df.loc['knowledge', 2019]
punctuality_init = X_df.loc['punctuality', 2019]
relevance_init = X_df.loc['relevance', 2019]
value_init = X_df.loc['value', 2019]

knowledge = widgets.FloatSlider(
    value=knowledge_init,
    min=knowledge_init - 0.1,
    max=knowledge_init + 0.1,
    step=0.01,
    description='knowledge',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.2f',
)

punctuality = widgets.FloatSlider(
    value=punctuality_init,
    min=punctuality_init - 0.1,
    max=punctuality_init + 0.1,
    step=0.01,
    description='punctuality',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.2f',
)

relevance = widgets.FloatSlider(
    value=relevance_init,
    min=relevance_init - 0.1,
    max=relevance_init + 0.1,
    step=0.01,
    description='relevance',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.2f',
)

value = widgets.FloatSlider(
    value=value_init,
    min=value_init - 0.1,
    max=value_init + 0.1,
    step=0.01,
    description='value',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.2f',
)

# slider_layout = widgets.VBox([knowledge, punctuality, relevance, value])

In [30]:
# import bqplot.pyplot as plt
# import pandas as pd

# X = np.array([
# 1,
# knowledge.value,
# punctuality.value,
# relevance.value,
# value.value
# ])

# outputs = pd.DataFrame({
# 'outcomes':['NKI', 'market_share'],
# 'value':[round(X@coef_NKI, 3), round(X@coef_share, 3)]
# })

# title = f'''
# What-if\n
# original NKI = {Y_df.loc['Nöjd Kund Index (NKI) allmänheten', 2019]}, 
# original market share = {Y_df.loc['Kollektivtrafikens marknadsandel', 2019]}\n
# estimated NKI = {outputs.iloc[0,1]},
# estimated market share = {outputs.iloc[1,1]})
# '''

# fig = px.bar(outputs, x='outcomes', y='value',
#              width=500, height=300, range_y = [0, 0.8], text='value', title=title)

# <center>Example Dashboard</center>

In [31]:
from ipywidgets import interact, HBox, VBox, interactive_output
import pandas as pd
import plotly.express as px

y_df = Y_df.loc[['Nöjd Kund Index (NKI) allmänheten', 'Kollektivtrafikens marknadsandel'],:]
y_df.index = ['NKI', 'market_share']

data_df = pd.concat([X_df, y_df], axis=0).unstack().reset_index()
data_df.columns = ['year', 'variable', 'value']
data_df['type'] = (['driver']*4 + ['outcome']*2 )*10
data_df['value'] = data_df['value'].apply(lambda x:round(x, 2))

In [32]:
# fig = px.line(data_df, x='year', y='value', color='variable',
#               width=800, height=500, range_y = [0, 0.8], line_dash='type',
#               category_orders={'variable':['NKI', 'market_share', 'knowledge', 'punctuality', 'relevance', 'value'],
#                               'type':['outcome', 'driver']},
#               title='Data history')
fig = px.line(data_df, x='year', y='value', color='variable',
              width=800, height=500, range_y = [0, 0.8], facet_row='type',
              category_orders={'variable':['NKI', 'market_share', 'knowledge', 'punctuality', 'relevance', 'value'],
                              'type':['outcome', 'driver']},
              title='Data history')

fig.update_layout(xaxis = {'nticks':10})
fig.show()

In [33]:

def plot(knowledge, punctuality, relevance, value):
    X = np.array([
    1,
    knowledge,
    punctuality,
    relevance,
    value
    ])
    
    outputs = pd.DataFrame({
    'outcomes':['NKI', 'market_share'],
    'value':[round(X@coef_NKI, 3), round(X@coef_share, 3)]
    })
    
    fig = px.bar(outputs, x='outcomes', y='value',
                width=500, height=400, range_y = [0, 0.8], text='value',
                title='Estimated outcomes')

    return fig.show()

out = widgets.interactive_output(plot, 
                                 {'knowledge': knowledge, 'punctuality': punctuality, 'relevance': relevance, 'value':value})

layout = widgets.Layout(display='flex',
                flex_flow='raw',
                align_items='center',
                width='100%')
display(HBox([VBox([knowledge, punctuality, relevance, value]), out], layout=layout))

knowledge = For most trips I make, I know how to go with the company.

punctuality = I can trust that I will arrive on time with the company.

relevance = I can use the company for most trips I make.

value (for money) = It is affordable to travel with the company.

----

## Experimental function
### From goal of outcomes, inversely estimate drivers to be achieved.

In [44]:
NKI_init = y_df.loc['NKI', 2019]
share_init = y_df.loc['market_share', 2019]

NKI = widgets.FloatSlider(
    value=NKI_init,
    min=NKI_init - 0.1,
    max=NKI_init + 0.1,
    step=0.01,
    description='NKI',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.2f',
)

share = widgets.FloatSlider(
    value=share_init,
    min=share_init - 0.1,
    max=share_init + 0.1,
    step=0.01,
    description='market share',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.2f',
)

In [45]:

def plot2(NKI, share):
    y = np.array([
    NKI,
    share
    ])

    inv = np.linalg.inv(coef.T @ coef) @ coef.T

    outputs = pd.DataFrame({
    'drivers':['knowledge', 'punctuality', 'relevance', 'value'],
    'value':(y@inv)[1:]
    })
    outputs['value'] = outputs['value'].apply(lambda x:round(x,3))
    outputs['value'] = outputs['value'].apply(lambda x:min(x,1.0))
    
    fig = px.bar(outputs, x='drivers', y='value',
                width=500, height=400, range_y = [0, 1.0], text='value',
                title='Drivers to be achieved')

    return fig.show()

out = widgets.interactive_output(plot2, 
                                 {'NKI': NKI, 'share': share})

layout = widgets.Layout(display='flex',
                flex_flow='raw',
                align_items='center',
                width='100%')
display(HBox([VBox([NKI, share]), out], layout=layout))